In [1]:
from tabuleiro import tabuleiro
from state import state
from numpy import Infinity
import re
from IPython.display import clear_output
import time
import copy

In [2]:
gameTabuleiro = tabuleiro()

In [3]:
pacman_inicial = [1,1,0]

In [4]:
fantasma_inicial = [9,9,0]

In [5]:
turno_inicial = True #Jogador

In [6]:
gamestate = state(tabuleiro(), pacman_inicial, fantasma_inicial, turno_inicial, 10)

In [7]:
def MinMax(state, moves, depth, turno):
    if turno:  # Se for turno do Max (Jogador Humano)
        best_score = float('-inf')
        best_move = None
    else:  # Se for turno do Min (Fantasma)
        best_score = float('inf')
        best_move = None
    if depth == 0 or state.isTerminal():
        score = state.get_score()
        return [None, score]
    # Itera pelos movimentos possíveis
    for move in moves:
        next_state = state.update(move)
        _, current_score = MinMax(next_state, next_state.movimentos_validos(), depth - 1, not turno)
        if turno:  # Maximiza para o Jogador Humano
            if current_score > best_score:
                best_score = current_score
                best_move = move
        else:  # Minimiza para o Fantasma
            if current_score < best_score:
                best_score = current_score
                best_move = move
    return [best_move, best_score]


In [8]:
def HillClimb(gamestate: state, depth, moves = []):
    if depth == 0:
        return [None, gamestate.get_distance()]
    moves = gamestate.movimentos_validos()
    melhor_distancia = gamestate.get_distance()
    melhorou = True
    melhor_movimento = moves[0]

         
    for move in moves:
        next_state = state.update(gamestate, move)
        
        _, distancia_nova = HillClimb(next_state, depth - 1)

        if distancia_nova < melhor_distancia:
            melhor_distancia = distancia_nova
            melhor_movimento = move
            

        


    return melhor_movimento, melhor_distancia

In [9]:
def obter_direcao():
    # Solicita a entrada do usuário
    escolha = input("Escolha uma direção (W = Norte, S = Sul, D = Leste (direita), A = Oeste, up = Subir de nível, down = Descer de nível) ou um andar para visualizar: ").strip().lower()

    # Expressões regulares para capturar as direções
    direcoes = {
        r'^(n|norte|north|w)$': 'Norte',
        r'^(s|sul|south)$': 'Sul',
        r'^(l|leste|east|direita|right|d)$': 'Leste',
        r'^(o|oeste|west|esquerda|left|a)$': 'Oeste',
        r'^(up|subir|subir de nivel|subir de nível|ascender)$': 'Subir de nível',
        r'^(down|descer|descer de nivel|descer de nível|descender)$': 'Descer de nível',
        r'^(1|andar 1| andar1)$': "Andar 1",
        r'^(2|andar 2| andar2)$': "Andar 2",
        r'^(3|andar 3| andar3)$': "Andar 3",
    }

    # Verifica se a escolha do usuário corresponde a alguma das expressões regulares
    for padrao, direcao in direcoes.items():
        if re.match(padrao, escolha):
            return direcao
    
    return
        

In [10]:
def jogador(state:state, direcao):
    while True:
        x, y, z = state.pacman
        # Ajusta a posição com base na direção
        if direcao == 'Norte':
            x -= 1  # Move para cima no plano 2D
        elif direcao == 'Sul':
            x += 1  # Move para baixo no plano 2D
        elif direcao == 'Leste':
            y += 1  # Move para a direita
        elif direcao == 'Oeste':
            y -= 1  # Move para a esquerda
        elif direcao == 'Subir de nível':
            z += 1  # Sobe de nível
        elif direcao == 'Descer de nível':
            z -= 1  # Desce de nível
        if [x,y,z] in state.movimentos_validos():
            return [x, y, z]
        elif direcao == "Andar 1":
            clear_output()
            state.tabuleiro.imprimir_tabuleiro(0)
            direcao = obter_direcao()
        elif direcao == "Andar 2":
            clear_output()
            state.tabuleiro.imprimir_tabuleiro(1)
            direcao = obter_direcao()
        elif direcao == "Andar 3":
            clear_output()
            state.tabuleiro.imprimir_tabuleiro(2)
            direcao = obter_direcao()
        else:
            print("Movimento inválido ou fora dos limites da matriz!")
            direcao = obter_direcao()
    

In [11]:
def rodar_jogo_hill_climb():
    gamestate.tabuleiro.imprimir_tabuleiro(0)
    while not gamestate.isTerminal():
        if gamestate.turno:
            movimento = jogador(gamestate, obter_direcao())
            time.sleep(1)
            clear_output()
            gamestate.update(movimento)
            gamestate.tabuleiro.imprimir_tabuleiro(movimento[2])
        else:
            copia = copy.deepcopy(gamestate)
            movimento = HillClimb(copia, 3, gamestate.movimentos_validos())
            gamestate.update(movimento[0])
            clear_output()
            print(movimento)
            
            gamestate.tabuleiro.imprimir_tabuleiro(movimento[0][2])

        gamestate.turno = not gamestate.turno

In [12]:
def rodar_jogo_min_max():
    gamestate.tabuleiro.imprimir_tabuleiro(0)
    while not gamestate.isTerminal():
        if gamestate.turno:
            movimento = jogador(gamestate, obter_direcao())
            time.sleep(1)
            clear_output()
            gamestate.update(movimento)
            gamestate.tabuleiro.imprimir_tabuleiro(movimento[2])
            
        else:
            copia = copy.deepcopy(gamestate)
            movimento = MinMax(copia, gamestate.movimentos_validos(), gamestate.Maxdepth, gamestate.turno)
            gamestate.update(movimento[0])
            clear_output()
            #print(movimento)
            
            
            gamestate.tabuleiro.imprimir_tabuleiro(movimento[0][2])
        gamestate.turno = not gamestate.turno

In [13]:
#rodar_jogo_min_max()

In [ ]:
rodar_jogo_hill_climb()

([1, 4, 0], 0)
■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
■ 0 0 0 F 0 0 0 0 1 ■
■ ■ 1 ■ 0 ■ ■ ■ 0 ■ ■
■ 1 1 ■ P 1 1 1 1 1 ■
■ 1 ■ ■ 1 ■ 1 ■ ■ 1 ■
■ 1 1 1 1 ■ 1 ■ 1 1 ■
■ 1 1 1 ■ ■ 1 ■ 1 ■ ■
■ 1 1 1 1 1 1 ■ 1 1 ■
■ 1 ■ ■ 1 ■ ■ ■ 1 ■ ■
■ 1 1 1 1 1 1 1 1 1 ■
■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
Movimento inválido ou fora dos limites da matriz!
Movimento inválido ou fora dos limites da matriz!
Movimento inválido ou fora dos limites da matriz!
Movimento inválido ou fora dos limites da matriz!
